In [1]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [2]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install sentencepiece --quiet
!pip install torch --quiet
!pip install googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org

In [3]:
import googletrans
from googletrans import Translator

In [4]:
translator = Translator()

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def translate_batch(texts, src_lang, tgt_lang):
    translated_texts = []
    for text in texts:
      translated = translator.translate(text, src=src_lang, dest=tgt_lang)
      translated_texts.append(translated.text)
    return translated_texts

def gpu_translation(sentences, src_lang, tgt_lang, batch_size=16, device='cuda'):
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    if device == 'cuda':
        print('on gpu')
        device = torch.device('cuda')
    else:
        print('on cpu')
        device = torch.device('cpu')
    
    translations = []
    with torch.no_grad():
      for i in range(0, len(sentences), batch_size):
          batch = sentences[i:i + batch_size]
          translations.extend(translate_batch(batch, src_lang, tgt_lang))
          print('translated: ', i)
    
    return translations


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
uploaded_df = pd.read_csv('/content/drive/My Drive/266_final_project/google_translated_data/df.csv')
src_texts = list(uploaded_df['english'].values)
new_df = uploaded_df.drop(columns="Unnamed: 0")
new_df['swahili_2'] = gpu_translation(src_texts, "en", "sw", batch_size=16, device='cuda')
new_df.to_csv("/content/drive/My Drive/266_final_project/google_translated_data/full_df.csv")

on gpu
translated:  0
translated:  16
translated:  32
translated:  48
translated:  64
translated:  80
translated:  96
